In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

In [ ]:
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
exit(0)

In [50]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os

from detectron2.config import get_cfg
from google.colab.patches import cv2_imshow
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.utils.logger import setup_logger
setup_logger()

In [ ]:
%cd /content/gdrive/MyDrive/Stone_Segmentation/
%ls

In [52]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 
predictor = DefaultPredictor(cfg)

def random_colour_masks(image):
  r = np.zeros_like(image).astype(np.uint8)
  g = np.zeros_like(image).astype(np.uint8)
  b = np.zeros_like(image).astype(np.uint8)
  r[image == 1], g[image == 1], b[image == 1] = [0,0,255]
  coloured_mask = np.stack([r, g, b], axis=2)
  return coloured_mask

In [53]:
im = cv2.imread("Stone_Data/train/28.jpg")
height = im.shape[0]
width = im.shape[1]
outputs = predictor(im)

mask =outputs["instances"].get("pred_masks").cpu().numpy()
boxes = outputs["instances"].get("pred_boxes").tensor.cpu().numpy()
area = []
for Mask in mask:
  pixels = cv2.countNonZero(cv2.UMat(np.array(Mask, dtype=np.uint8)))
  area_cm = (0.0007000434027776013)*pixels
  area.append(area_cm)
larget_stone_num = area.index(max(area))
rgb_mask = random_colour_masks(mask[larget_stone_num])
xmin,ymin,xmax,ymax = boxes[larget_stone_num]
img = cv2.addWeighted(im, 1, rgb_mask, 0.5, 0)
cv2.putText(img,"Larger_Stone,Size:"+str("{:.2f}".format(area[larget_stone_num])) +"cm2",(int(xmin),int(ymin)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255))
cv2_imshow(img)

Output hidden; open in https://colab.research.google.com to view.